In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.0 chat - Multi-turn

* [Google Gen AI SDKs](https://cloud.google.com/vertex-ai/generative-ai/docs/sdks/overview)
  * The new Google Gen AI SDK provides a unified interface to Gemini 2.0 through both the Gemini Developer API and the Gemini API on Vertex AI. With a few exceptions, code that runs on one platform will run on both.
  * This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

* [google-genai 0.3.0](https://pypi.org/project/google-genai/)
* [Google Gen AI SDK](https://googleapis.github.io/python-genai/)
* [python-genai : Github](https://github.com/googleapis/python-genai)

## Set configuration

### Package Install
* [python-genai](https://googleapis.github.io/python-genai/)

In [1]:
%pip install --upgrade --user --quiet google-genai

### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [3]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-2.0-flash-exp" # @param {type:"string"}

### Import libraries

In [4]:
import base64

from google import genai
from google.genai.chats import Chats

from google.genai.types import (
    Content,
    FunctionDeclaration,
    GenerateContentConfig,
    GenerateContentResponse,
    GoogleSearch,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    VertexAISearch,
)


### Initalize client

* [genai.Client](https://googleapis.github.io/python-genai/genai.html#genai.client.Client)

In [5]:
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION
)

### Helper function

In [6]:
def generate_content_config():
  """
  Method to configure generation config.

  """
  # https://googleapis.github.io/python-genai/genai.html#genai.types.SafetySettingDict
  safety_settings = [SafetySetting(category="HARM_CATEGORY_HATE_SPEECH",threshold="OFF"),
                      SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT",threshold="OFF"),
                      SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT",threshold="OFF"),
                      SafetySetting(category="HARM_CATEGORY_HARASSMENT",threshold="OFF")]

  google_search_tool = Tool(google_search=GoogleSearch())

  # https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
  generate_content_config = GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    top_k=20,
    candidate_count=1,
    seed = 5,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = safety_settings,
    tools = [google_search_tool]
  )

  return generate_content_config


def chat_gemini_2_0(chat:Chats, query:str) -> GenerateContentResponse:

  # https://googleapis.github.io/python-genai/genai.html#genai.chats.Chats.create
  response = chat.send_message(query)
  return response


### Execution

In [8]:
gen_config = generate_content_config()
# https://googleapis.github.io/python-genai/genai.html#genai.chats.Chats.create
chat = client.chats.create(model=MODEL_NAME, config=gen_config)

while True:
  query = input('사용자: ')

  if query == '종료': break

  # chat_history = get_chat_history(chat)

  prompt = f"""
  당신은 최신 정보를 포함한 다양한 사용자의 질문에 답을 해주는 AI Assistant 입니다.
  아래 질문에 대해서 친절하게 10줄 이내로 답해주세요.

  질문 : {query}
  """

  response = chat_gemini_2_0(chat, prompt)
  display(Markdown(f"AI Agent : {response.text}"))
  print(f"------------------------------------ ")


사용자: 안녕하세요.


AI Agent : 안녕하세요! 무엇을 도와드릴까요? 궁금한 점이나 필요한 정보가 있으시면 언제든지 물어보세요. 제가 아는 선에서 최대한 친절하고 자세하게 답변해 드리겠습니다.


------------------------------------ 
사용자: 최근 한국의 정치 상황에 대해서 설명해주세욧.


AI Agent : 최근 한국 정치 상황은 매우 혼란스럽고 역동적입니다. 2024년 12월 초 윤석열 대통령이 계엄령을 선포했다가 철회하는 사건이 발생하면서 정치적 위기가 고조되었습니다. 이로 인해 대통령에 대한 탄핵 소추안이 국회에서 가결되었고, 현재 헌법재판소에서 탄핵 심판이 진행 중입니다.

주요 정치 쟁점으로는 대통령의 계엄령 시도, 여야 간의 극심한 대립, 경제 불안정 등이 있습니다. 윤석열 대통령의 지지율은 매우 낮은 수준이며, 여당과 야당 모두 내부 분열을 겪고 있는 상황입니다.

이러한 정치적 불안정은 경제에도 영향을 미쳐 원화 가치가 하락하고 금융 시장이 혼란스러워졌습니다. 또한, 북한의 핵 위협이 고조되는 상황에서 국내 정치 불안정은 외교에도 부정적인 영향을 미칠 수 있다는 우려가 제기되고 있습니다.

정리하자면, 한국 정치는 현재 대통령 탄핵 심판과 여야 갈등으로 인해 매우 불안정한 상황이며, 이로 인한 경제 및 외교적 파장도 우려되는 상황입니다.


------------------------------------ 
사용자: 한국의 정치 상황이 내년 초반에 부동산 시장에 어떤 영향을 줄까요 ?


...

------------------------------------ 
사용자: 올해 하반기에 AI 주요 이슈 뉴스에 대해서 10가지 알려주세요.


AI Agent : 올해 하반기 인공지능(AI) 분야의 주요 이슈 10가지 뉴스는 다음과 같습니다:

1.  **생성형 AI 모델 경쟁 심화:** 오픈AI의 GPT-4를 비롯해 구글의 제미나이, 메타의 라마 등 다양한 생성형 AI 모델들이 출시되어 경쟁이 더욱 치열해졌습니다.
2.  **AI 윤리 및 규제 논의 활발:** AI 기술의 발전 속도에 발맞춰 AI 윤리 문제와 규제 필요성에 대한 논의가 국제적으로 활발하게 진행되었습니다. AI의 편향성, 투명성, 책임 소재 등이 주요 쟁점으로 부각되었습니다.
3.  **AI 저작권 논란 가열:** AI가 생성한 콘텐츠에 대한 저작권 문제가 중요한 이슈로 떠올랐습니다. AI 학습 데이터와 생성물의 저작권 소유권에 대한 법적 논의가 활발히 진행 중입니다.
4.  **AI 반도체 시장 경쟁 격화:** AI 모델 구동에 필요한 고성능 반도체에 대한 수요가 증가하면서 엔비디아를 필두로 삼성전자, TSMC 등 반도체 기업들의 경쟁이 심화되었습니다.
5.  **AI 의료 분야 발전 가속화:** AI 기술을 활용한 질병 진단, 신약 개발 등 의료 분야에서 AI의 역할이 확대되고 있으며, 다양한 의료 AI 솔루션이 출시되었습니다.
6.  **AI 교육 분야 도입 확대:** AI 기반 학습 플랫폼, 튜터링 서비스 등 AI 기술이 교육 분야에 도입되며 개인 맞춤형 교육 솔루션이 늘어났습니다.
7.  **AI 챗봇 및 가상 비서 진화:** 자연어 처리 기술이 발전하면서 더욱 정교한 AI 챗봇과 가상 비서 서비스가 등장했습니다. 일상생활에서 AI를 활용하는 사례가 늘어났습니다.
8.  **AI 일자리 변화 우려:** AI 자동화로 인해 일부 직업이 사라질 수 있다는 우려가 제기되었고, AI 시대에 필요한 새로운 기술 및 교육에 대한 관심이 높아졌습니다.
9.  **오픈소스 AI 모델 확산:** 메타의 라마 등 오픈소스 AI 모델이 공개되면서 AI 기술 개발 장벽이 낮아지고 다양한 분야에서 AI 응용이 확산되었습니다.
10. **AI 안전성 문제 대두:** AI 기술 오남용 및 사이버 공격에 대한 우려가 커지면서 AI 안전성에 대한 논의와 기술 개발에 대한 중요성이 강조되었습니다.


------------------------------------ 
사용자: 의료분야에서는 어떤 측면에서 활용되나요 ?


AI Agent : 인공지능(AI)은 의료 분야에서 다양한 측면으로 활용되어 의료 서비스의 질을 향상시키고 효율성을 높이는 데 기여하고 있습니다. 주요 활용 분야는 다음과 같습니다:

1.  **질병 진단:** AI는 의료 영상(X-레이, MRI, CT 스캔 등)을 분석하여 종양, 질병 징후 등을 조기에 발견하는 데 도움을 줍니다.
2.  **신약 개발:** AI는 방대한 데이터를 분석하여 신약 후보 물질을 발굴하고 임상 시험 기간을 단축하는 데 기여합니다.
3.  **맞춤형 치료:** AI는 환자 데이터를 기반으로 개인에게 최적화된 맞춤형 치료법을 제시하는 데 활용됩니다.
4.  **수술 보조:** AI 기반 로봇 수술 시스템은 정밀한 수술을 가능하게 하고 수술 시간 단축 및 합병증 위험을 줄이는 데 도움을 줍니다.
5.  **환자 모니터링:** AI는 웨어러블 기기 등을 통해 환자의 건강 상태를 실시간으로 모니터링하고 이상 징후를 조기에 감지하는 데 사용됩니다.
6.  **의료 기록 관리:** AI는 의료 기록을 정리하고 분석하여 의료진의 업무 효율성을 높이는 데 기여합니다.
7.  **원격 의료:** AI 챗봇 및 가상 비서는 환자와 의료진 간의 원활한 소통을 돕고 원격 진료 서비스를 제공하는 데 활용됩니다.
8.  **감염병 예측:** AI는 감염병 확산 경로를 예측하고 예방 전략을 수립하는 데 활용됩니다.
9.  **의료진 교육:** AI 기반 시뮬레이션 및 교육 프로그램은 의료진의 역량 강화를 돕습니다.
10. **만성질환 관리:** AI는 만성 질환 환자의 건강 데이터를 분석하여 맞춤형 관리 프로그램을 제공하고 질병 악화를 예방하는 데 기여합니다.

이 외에도 AI는 의료 분야에서 다양한 가능성을 보여주며 지속적인 발전이 기대됩니다.


------------------------------------ 
사용자: 종료
